# 패키지 와 데이터 로드

In [2]:
import time
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from sklearn_extra.cluster import KMedoids

from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.mixture import GaussianMixture

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import estimate_bandwidth
from sklearn.cluster import KMeans
from sklearn.cluster import SpectralClustering
from sklearn.cluster import MeanShift
from sklearn.cluster import AgglomerativeClustering
# from konlpy.tag import Komoran

# import pandas_datareader as pdr
from datetime import datetime

from scipy.cluster.hierarchy import linkage 
from scipy.cluster.hierarchy import dendrogram 

from tslearn.clustering import TimeSeriesKMeans
from tslearn.metrics import dtw

import scipy.spatial.distance as ssd

# from gensim.models import Word2Vec as w_to_v
import warnings
warnings.filterwarnings(action='ignore')


In [3]:
#df.to_pickle('C:/Users/Dohyung/Desktop/데이터 마이닝/데이터 마이닝_프로젝트/ipynb/name,code,x1,x2,x3,x,x_nonumber,morph,morph_sum,labels_kse,labels_fics.fkl')
#df = pd.read_pickle('./name,code,x1,x2,x3,x,x_nonumber,morph,morph_sum,labels_kse,labels_fics.fkl')
df = pd.read_csv('../data/df.csv', encoding = 'euc-kr')
df_price = pd.read_pickle('../fkl/price_normal.fkl')
df_dtw_matrix = pd.read_pickle('../fkl/dtw_matrix.fkl')

df_finance = pd.read_pickle('../fkl/financial_i_log_categoric.fkl')
df_finance = pd.read_csv('../data/financial_i.csv', encoding = 'euc-kr') #직접 수정본, log 스케일

# df_word = pd.read_csv('../data/khaiii_parsing.csv', encoding = 'euc-kr') #fnguide
df_dart = pd.read_csv('../data/dart.csv', encoding = 'euc-kr')
df_dart_origin = pd.read_pickle('../fkl/dart_crawling_result.fkl')

# 결측값 제거
df_price = df_price.dropna(axis = 0)

FileNotFoundError: [Errno 2] No such file or directory: '../data/financial_i.csv'

In [ ]:
# kse : 한국 거래소에 의한 산업 분류
# 어떤 분류들이 있는지 확인
df['labels_kse'].unique()

In [ ]:
#Kmeans
#재무 제표에 대한 클러스터링
finance_data = df_finance[["PER", "PBR", "PCR", "EV/EBITDA", "Dividend"]]
clst = KMeans(n_clusters=5)
clst.fit(finance_data)


clusters = clst.labels_.tolist()

centers = clst.cluster_centers_

In [ ]:
# PCA 차원 축소
%matplotlib inline
using_pca = PCA(n_components=3)
using_pca.fit(finance_data)
pca= using_pca.transform(finance_data)
centers_trans= using_pca.transform(centers)

xs, ys, zs = pca[:, 0], pca[:, 1], pca[:, 2]

df_plot = pd.DataFrame(dict(x=xs, y=ys, z=zs, label=clusters))
groups = df_plot.groupby('label')


fig = plt.figure(figsize=(10, 9))
ax = fig.add_subplot(111, projection = '3d')
ax.margins(0.05)
colors = {0:'r', 1: 'b', 2: 'k', 3: 'm', 4: 'c', 5: 'g', 6: 'w'}
for idx, group in groups:
    ax.plot(group.x, group.y, group.z ,marker='o', linestyle='', ms=5, mec='none')
    ax.set_aspect('auto')
    ax.tick_params(        axis= 'x',
        which='both',
        bottom='off',
        top='off',
        labelbottom='off')
    ax.tick_params(        axis= 'y',
        which='both',
        left='off',
        top='off',
        labelleft='off')
    ax.tick_params(        axis= 'z',
        which='both',
        left='off',
        top='off',
        labelleft='off')



ax.plot(centers_trans[:,0],centers_trans[:,1],centers_trans[:,2], c='y', zorder=100, marker='d', linestyle='')
plt.show()

# 토큰화

In [ ]:
# !토크나이징
# 핵심

morphs=[]

komoran = Komoran(userdic='./my_dict.txt')
string_list = []
for i in range(0,200):
    noun = komoran.nouns(df_dart.iloc[i]['x2']) #morphs 로 하면, 형태소 단위로 다 쪼갬
    morphs.append(noun)
for j in range(0,200):
    every_morphs = komoran.pos(df_dart.iloc[j]['x2'])
    english = []
    for k in every_morphs:
        if k[1] == 'SL':
            english.append(k[0])

    morphs[j] += english

df_dart["morph"]= morphs






for i in range(0,200):
    sum_string = ""
    for j in range(0,len(df_dart.iloc[i]["morph"])):
        sum_string+= (df_dart.iloc[i]["morph"][j]+ " ")
    string_list.append(sum_string)
    
df_dart["morph_sum"] = string_list
df_dart

In [ ]:
with_dart = []
df_dartcopy = df_dart.copy()
df_copy = df.copy()

for i in range(0,200):
    A = df_dartcopy.iloc[i]["morph_dart"]
    B = df_copy.iloc[i]["morph_sum"]
    with_dart.append(A+B)
    
df["morph_sumdart"] = with_dart
df

# 기업 개요 데이터 클러스터링

In [ ]:
#TFidf 변환 및 KMeans 학습

countvectorizer = CountVectorizer(stop_words=['1월', '1일','2월','3월','4월', '5월', '6월', '7월', '8월', '9월', '10월', '11월', '12월', 'M', 'co', 'company', 'global', 'inc', '가능', '가능', '가치', '각종', 
                                              '감소', '강화', '개', '개선', '개시', '개요', '거점', '결과', '결정', '경영', 
                                              '경우', '경쟁력', '계속', '계획', '공시', '과정', '관련', '관리', '구분', 
                                              '구성', '구조', '구축', '국가', '국내', '국내외', '규모', '극대', '글로벌', 
                                              '기간', '기능', '기대', '기록', '기반', '기업', '기존', '기준', '기초',
                                              '기타', '기회', '내용', '노력', '누적', '능력', '다각', '다수', '다양', '다음',
                                              '단계', '단위', '단일', '달성', '담당', '당기', '당사', '대비', '대상', '대응',
                                              '도약', '도입', '동기', '동력', '동사', '동시', '드', '등', '마련', '매출', 
                                              '매출액', '목적', '목표', '및', '바탕', '반기', '반영', '방식', '법인', '변경',
                                              '변동', '변화', '별도', '보수', '보호', '부가', '부문', '부분', '분기', '분류',
                                              '분야', '비용', '비중', '사업', '사업장', '사용', '사항', '상기', '상반기', 
                                              '상승', '상장', '상호', '상황', '선도', '설계', '설립', '설비', '성격', '성장',
                                              '세계', '소유', '수요', '수익', '수준', '수출', '수행', '시작', '시장', '시행',
                                              '실적', '아래', '안정', '업무', '업체', '역량', '역할', '연간', '연결', '영역', 
                                              '영업', '영위', '영향', '예상', '예정', '완료', '요구', '요약', '요인', '우수', 
                                              '우위', '운영', '위주', '위치', '유가증권', '유지', '의', '이상', '이익', '이하', 
                                              '이후', '일반', '일부', '임대', '입지', '자금', '자본', '자산', '자체', '작성', 
                                              '장기', '장벽', '장점', '재무', '적극', '적용', '전개', '전년', '전년 동기', 
                                              '전략', '전망', '전문', '전반', '전체', '점유', '제고', '제외', '제품', '조정', '조직', 
                                              '종속', '종합', '주', '주도', '주력', '주식', '주식회사', '주요', '준비', '중심',
                                              '중요', '증가', '증대', '지배', '지분', '지속', '지역', '지원', '지위', '지주',
                                              '지주회사', '진행', '집중', '차별', '차지', '참고', '참여', '참조', '창출',
                                              '채널', '처리', '체계', '체제', '최고', '최근', '최대', '최소', '최초',
                                              '추가', '추세', '추진', '충족', '투자', '특성', '특수', '특징', '평가',
                                              '포함', '필요', '항목', '해당', '해외', '핵심', '향상', '향후', '현재', '현지',
                                              '현황', '형태', '확대', '확보', '확장', '활동', '활용', '회사', '효율'], 
                                  ngram_range=(1,2), max_df = 0.4, min_df = 0.05, lowercase=True, max_features = 550) #0.4 0.05
tfidftransformer = TfidfTransformer()

data = df["morph_sumdart"].copy()
data_trans = tfidftransformer.fit_transform(countvectorizer.fit_transform(data))


clst = KMeans(n_clusters=10, random_state=0)
clst.fit(data_trans)


clusters = clst.labels_.tolist()

centers = clst.cluster_centers_

In [ ]:
print(data_trans.shape)

In [ ]:
countvectorizer.stop_words_

In [ ]:
countvectorizer.vocabulary_

In [ ]:
#계층적 클러스터링(강의버젼)

hac = linkage(tfidf_matrix, 'ward') #single, average, complete, weighted, centroid, median, ward
plt.figure(figsize=(10, 7))
labelList = range(1,201)
dendrogram(hac,
            orientation='top',
            labels=labelList,
            distance_sort='descending',
            show_leaf_counts=True)
plt.show()

In [ ]:
# PCA 차원 축소
#%matplotlib notebook
using_pca = PCA(n_components=3)
using_pca.fit(data_trans.toarray())
pca= using_pca.transform(data_trans.toarray())
centers_trans= using_pca.transform(centers)

xs, ys, zs = pca[:, 0], pca[:, 1], pca[:, 2]

df_plot = pd.DataFrame(dict(x=xs, y=ys, z=zs, label=clusters))
groups = df_plot.groupby('label')


fig = plt.figure(figsize=(10, 9))
ax = fig.add_subplot(111, projection = '3d')
ax.margins(0.05)
colors = {0:'r', 1: 'b', 2: 'k', 3: 'm', 4: 'c', 5: 'g', 6: 'w'}
for idx, group in groups:
    ax.plot(group.x, group.y, group.z ,marker='o', linestyle='', ms=5, mec='none')
    ax.set_aspect('auto')
    ax.tick_params(        axis= 'x',
        which='both',
        bottom='off',
        top='off',
        labelbottom='off')
    ax.tick_params(        axis= 'y',
        which='both',
        left='off',
        top='off',
        labelleft='off')
    ax.tick_params(        axis= 'z',
        which='both',
        left='off',
        top='off',
        labelleft='off')



ax.plot(centers_trans[:,0],centers_trans[:,1],centers_trans[:,2], c='b', zorder=100, marker='d', linestyle='')
plt.show()

In [ ]:
# 데이터 프레임 형태로 클러스터 결과 확인

df["cluster"]= clusters

df_groupby_cluster = df.groupby('cluster')
#result = dict(list((df_groupby_cluster['name'])))

groupby_list = df.groupby('cluster')['labels_kse'].apply(list)

for i in groupby_list:
    print(i)
    print('\n')


In [ ]:
# 데이터 프레임 형태로 클러스터 결과 확인

df["cluster"]= clusters

df_groupby_cluster = df.groupby('cluster')
#result = dict(list((df_groupby_cluster['name'])))

groupby_list = df.groupby('cluster')['name'].apply(list)

for i in groupby_list:
    print(i)
    print('\n')

In [ ]:
#LDA 차원축소
lda = LinearDiscriminantAnalysis(n_components=3)

#학습
lda.fit(data_trans.toarray(), clusters)
#변형
data_lda = lda.transform(data_trans.toarray())
centers_trans= lda.transform(centers)

#DataFrame 생성
lda_columns=['lda_component_1','lda_component_2','lda_component_3']
DF_lda = pd.DataFrame(data_lda, columns=lda_columns)
DF_lda['target']=clusters


#축 지정
xs, ys, zs = data_lda[:, 0], data_lda[:, 1], data_lda[:, 2]

df_plot = pd.DataFrame(dict(x=xs, y=ys, z=zs, label=clusters))
groups = df_plot.groupby('label')

markers=['^', 's', 'o','d','*']
fig = plt.figure(figsize=(10, 9))
ax = fig.add_subplot(111, projection = '3d')
ax.margins(0.05)
colors = {0:'r', 1: 'b', 2: 'k', 3: 'm', 4: 'c', 5: 'g', 6: 'w'}
for idx, group in groups:
    ax.plot(group.x, group.y, group.z ,marker='o', linestyle='', ms=5, mec='none' )
    ax.set_aspect('auto')
    ax.tick_params(        axis= 'x',
        which='both',
        bottom='off',
        top='off',
        labelbottom='off')
    ax.tick_params(        axis= 'y',
        which='both',
        left='off',
        top='off',
        labelleft='off')
    ax.tick_params(        axis= 'z',
        which='both',
        left='off',
        top='off',
        labelleft='off')



ax.plot(centers_trans[:,0],centers_trans[:,1],centers_trans[:,2], c='y', zorder=100, marker='d', linestyle='')
plt.show()



# 주가 변동 데이터 클러스터링

## 클러스터링 알고리즘 선택

In [ ]:
#시계열 KMeans
n_clusters = 20
tsclst = TimeSeriesKMeans(n_clusters = n_clusters, metric = 'euclidean') #'euclidean', 'dtw'
tsclst.fit(df_price)

clusters = tsclst.labels_.tolist()

centers = tsclst.cluster_centers_

In [ ]:
len(tsclst.labels_)

In [ ]:
%matplotlib notebook
#pca 차원축소
using_pca = PCA(n_components=3)
using_pca.fit(df_price)
pca= using_pca.transform(df_price)
centers_trans= using_pca.transform(centers.reshape(n_clusters,248))

xs, ys, zs = pca[:, 0], pca[:, 1], pca[:, 2]

df_plot = pd.DataFrame(dict(x=xs, y=ys, z=zs, label=clusters))
groups = df_plot.groupby('label')


fig = plt.figure(figsize=(10, 9))
ax = fig.add_subplot(111, projection = '3d')
ax.margins(0.05)
colors = {0:'r', 1: 'b', 2: 'k', 3: 'm', 4: 'c', 5: 'g', 6: 'w',-1:'y'}
for idx, group in groups:
    ax.plot(group.x, group.y, group.z ,marker='o', linestyle='', ms=5, mec='none')
    ax.set_aspect('auto')
    ax.tick_params(        axis= 'x',
        which='both',
        bottom='off',
        top='off',
        labelbottom='off')
    ax.tick_params(        axis= 'y',
        which='both',
        left='off',
        top='off',
        labelleft='off')
    ax.tick_params(        axis= 'z',
        which='both',
        left='off',
        top='off',
        labelleft='off')



ax.plot(centers_trans[:,0],centers_trans[:,1],centers_trans[:,2], c='y', zorder=100, marker='d', linestyle='')
plt.show()

In [ ]:
df_price["cluster"] = clusters
index = []
cluster0 = df_price.query("cluster==2")
cluster0 = cluster0.drop("name", axis=1, errors = 'ignore')
cluster0 = cluster0.drop("cluster", axis=1, errors = 'ignore')
for i in range(0, len(cluster0)):
    sns.lineplot(y = cluster0.iloc[i], x = list(range(248)))


In [ ]:
centers_price = centers.reshape(n_clusters,248)
for i in range(0, n_clusters):
    sns.lineplot(y = centers_price[2], x = list(range(248)))
    

In [ ]:
# 데이터 프레임 형태로 클러스터 결과 확인

df_price["cluster"]= clusters
df_price["name"]= df_price.index

df_groupby_cluster = df_price.groupby('cluster')
#result = dict(list((df_groupby_cluster['name'])))

groupby_list = df_price.groupby('cluster')["name"].apply(list)

for i in groupby_list:
    print(i)
    print('\n')
df_price = df_price.drop("name",axis=1, errors = 'ignore')
df_price = df_price.drop("cluster",axis=1, errors = 'ignore')

In [ ]:
df_price